# Combining Data from Database Into CSV

In [5]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from config import login

In [6]:
# Establishing connection

db_url = 'postgresql://' + login + '@localhost:5432/movie_db'
engine = create_engine(db_url)
connection = engine.connect()

In [7]:
tag_query = f"select movie_id, tag from tags;"

tags = pd.read_sql(tag_query, connection)

len(tags)

1093360

In [10]:
movies_bag = pd.read_csv("big_movies_csv/movies_bag.csv")

In [11]:
movies_bag.head()

,movie_id,title,bag_of_words
0,1,Toy Story (1995),adventure animation children comedy fantasy ca...
1,2,Jumanji (1995),adventure children fantasy robinwilliams anima...
2,3,Grumpier Old Men (1995),comedy romance
3,4,Waiting to Exhale (1995),comedy drama romance
4,5,Father of the Bride Part II (1995),comedy


In [14]:
titanic = movies_bag["bag_of_words"].where(movies_bag['movie_id'] == 1721)

In [19]:
print(movies_bag['bag_of_words'][1655])

drama romance jamescameron katewinslet leonardodicaprio nudity(topless-notable) nudity(topless) oscar(bestcinematography) oscar(bestdirecting) oscar(bestpicture) atmospheric basedonatruestory bittersweet catastrophe chickflick disaster drama epic historical love lovestory music overrated periodpiece realistic romance romantic sadending sentimental shipwreck survival timetravel truestory


In [8]:
# Iterrows to convert each tag to lowercase without spaces
for index, row in tags.iterrows():
    tag_converted = row['tag'].lower().replace(" ", "")
    tags.iloc[index, 1] = tag_converted

tags.head()

KeyboardInterrupt: 

In [ ]:
# Now we need to combine the tags into one column based on movie_id

# First create a new dataframe to store the tags and movie_ids
tags_combined = pd.DataFrame(columns=["movie_id", "tags"])
tags_combined.head()

In [ ]:
# Iterrows for tags df
for index, row in tags.iterrows():
    # Check if movie_id already exists in tags_combined
    movie_exist = tags_combined.loc[tags_combined['movie_id']==row['movie_id'], :]
    #movie_exist.head()
    if movie_exist.empty:
        # Add new movie row
        new_row = {'movie_id': row['movie_id'], 'tags': row['tag']}
        #print (new_row)
        tags_combined = tags_combined.append(new_row, ignore_index=True)
    else:
        #print("Movie ID exists! Add to row")
        
        # Combine tags
        new_tags = movie_exist['tags'] + " " + row['tag']
        # Update row
        tags_combined.loc[tags_combined['movie_id']==row['movie_id'], 'tags'] = new_tags
        
tags_combined.head()

In [158]:
# Finding movie_id that have distinct count of tags >= 4 (Jeremy)

movie_sql_df = []

for x in range(len(tags_combined)):
    r = tags_combined['tags'][x].split(" ")
    if len(r) >= 2:
       movie_sql_df.append(tags_combined['movie_id'][x])
    else:
        pass

move_df = pd.DataFrame({"movie_id": movie_sql_df})

In [21]:
# Save combined_tags to CSV
tags_combined.to_csv("big_movies_csv/tags_combined.csv", index=False)

In [24]:
# Read genres from DB so we can merge genres with the tags

genre_query = f"select movie_id, genre from movies;"

genre = pd.read_sql(genre_query, connection)

genre.head()

,movie_id,genre
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


In [25]:
# Iterrows to convert each genre to lowercase, swap | with a space
for index, row in genre.iterrows():
    genre_converted = row['genre'].lower().replace("|", " ")
    genre.iloc[index, 1] = genre_converted
    #print (index, row['genre'], genre_converted)
    
genre.head()

,movie_id,genre
0,1,adventure animation children comedy fantasy
1,2,adventure children fantasy
2,3,comedy romance
3,4,comedy drama romance
4,5,comedy


In [26]:
# Merge genre and tags_converted on movie_id

combined_df = genre.merge(tags_combined, on="movie_id", how="outer").fillna("")
combined_df.head()

,movie_id,genre,tags
0,1,adventure animation children comedy fantasy,cartoon disney pixar timallen tomhanks adventu...
1,2,adventure children fantasy,robinwilliams animals badcgi boardgame family ...
2,3,comedy romance,
3,4,comedy drama romance,
4,5,comedy,


In [27]:
# Create new bag_of_words df that combines genre and tags

bag_of_words = pd.DataFrame(columns=["movie_id", "bag_of_words"])
bag_of_words.head()

,movie_id,bag_of_words


In [28]:
# Iterrows for combined_df
for index, row in combined_df.iterrows():
    # Combine genre and tags
    bag = row['genre'] + " " + row['tags']
    
    new_row = {'movie_id': row['movie_id'], 'bag_of_words': bag}
    
    # Append row
    bag_of_words = bag_of_words.append(new_row, ignore_index=True)
        
bag_of_words.head()

,movie_id,bag_of_words
0,1,adventure animation children comedy fantasy ca...
1,2,adventure children fantasy robinwilliams anima...
2,3,comedy romance
3,4,comedy drama romance
4,5,comedy


In [29]:
# Save bag_of_words to CSV
bag_of_words.to_csv("big_movies_csv/bag_of_words.csv", index=False)

In [30]:
# Read movie title from DB so we can separate title from year, then merge with bag_of_words

movie_query = f"select movie_id, title from movies;"

movies = pd.read_sql(movie_query, connection)

movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [31]:
# Merge movies and bag_of_words on movie_id

movies_bag = movies.merge(bag_of_words, on="movie_id", how="outer").fillna("")
movies_bag.head()

,movie_id,title,bag_of_words
0,1,Toy Story (1995),adventure animation children comedy fantasy ca...
1,2,Jumanji (1995),adventure children fantasy robinwilliams anima...
2,3,Grumpier Old Men (1995),comedy romance
3,4,Waiting to Exhale (1995),comedy drama romance
4,5,Father of the Bride Part II (1995),comedy


In [32]:
# Save movies_bag to CSV
movies_bag.to_csv("big_movies_csv/movies_bag.csv", index=False)

## Split the title and year

In [33]:
# Create new movie_year df so we can split title and year

movie_year = pd.DataFrame(columns=["movie_id", "title", "year"])
movie_year.head()

,movie_id,title,year


In [34]:
# Import regular expression because there are titles with (words) in the name and we need to make sure 
# we split it at (year)
import re

In [35]:
# Iterrows for movies
for index, row in movies.iterrows():
    # Combine genre and tags
    to_split = row['title']
    title_split = re.split(r'[ ](?=\([1-2][0-9][0-9][0-9])', to_split)
    title = title_split[0]
    if len(title_split)>1:
        year = title_split[1].replace("(", "").replace(")", "") # Then strip () from string
    else:
        print (f"No year for {row['title']}")
        year = ""
    
    new_row = {'movie_id': row['movie_id'], 'title': title, 'year': year}
    #print(new_row)
    
    # Append row
    movie_year = movie_year.append(new_row, ignore_index=True)
    
    #if index == 3:
    #    break
        
movie_year.head()

No year for Skokie (1981)
No year for Deadly Advice(1994)
No year for Millions Game, The (Das Millionenspiel)
No year for Me and the Colonel (1958)
No year for On the Double (1961)
No year for Here Comes Peter Cottontail (1971)
No year for Enchanted World of Danny Kaye: The Emperor's New Clothes, The (1972)
No year for Terrible Joe Moran
No year for The Court-Martial of Jackie Robinson
No year for In Our Garden
No year for Stephen Fry In America - New World
No year for Two: The Story of Roman & Nyro
No year for A Year Along the Abandoned Road
No year for Body/Cialo
No year for Polskie gówno
No year for The Third Reich: The Rise & Fall
No year for La vendetta dei barbari
No year for My Own Man
No year for Moving Alan
No year for Zone 261
No year for Michael Laudrup - en Fodboldspiller
No year for Honky
No year for Blueberry Hill
No year for Pleasure Palace
No year for Terror on the 40th Floor
No year for My Boyfriend's Back
No year for One Night Only
No year for Doli Saja Ke Rakhna
No y

,movie_id,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [36]:
movie_year.tail()

,movie_id,title,year
62418,209157,We,2018
62419,209159,Window of the Soul,2001
62420,209163,Bad Poems,2018
62421,209169,A Girl Thing,2001
62422,209171,Women of Devil's Island,1962


In [37]:
# Test
test_string = "Babylon 5"
title_split = re.split(r'[ ](?=\([1-2][0-9][0-9][0-9])', test_string)
title = title_split[0]
if len(title_split)>1:
    year = title_split[1]
else:
    year = 0
print(year)

0


In [38]:
# Merge movie_year with bag_of_words on movie_id

movies_bag = movie_year.merge(bag_of_words, on="movie_id", how="outer").fillna("")
movies_bag.head()

,movie_id,title,year,bag_of_words
0,1,Toy Story,1995,adventure animation children comedy fantasy ca...
1,2,Jumanji,1995,adventure children fantasy robinwilliams anima...
2,3,Grumpier Old Men,1995,comedy romance
3,4,Waiting to Exhale,1995,comedy drama romance
4,5,Father of the Bride Part II,1995,comedy


In [159]:
the_main_df = pd.merge(movies_bag, move_df, how="right", on=["movie_id", "movie_id"])
the_main_df.to_csv("big_movies_csv/the_main_df.csv", index=False)

In [57]:
# Make sure to close the connection

connection.close()